In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
# %matplotlib widget
%matplotlib inline
import matplotlib.font_manager as fm

In [ ]:
# 시각화 한글 설정
fonts = fm.findSystemFonts()
nanum_path = None
for font in fonts:
    if font.endswith('NanumGothic.ttf'):
        nanum_path = font
        break
if nanum_path == None:
    print(f'나눔 폰트를 설치해야 합니다!')
    print(f'!apt install -qq -y fonts-nanum*')
else:
    print(f'나눔 폰트 경로: {nanum_path}')
    nanum_prop = fm.FontProperties(fname=nanum_path)

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.cifar10.load_data()
images = np.concatenate((train_images, test_images))
labels = np.concatenate((train_labels, test_labels))
print(f'학습에 사용할 이미지는 {len(train_images):,}개 입니다.')
print(f'학습한 후 테스트(검증)에 사용할 이미지는 {len(test_images):,}개 입니다.')

In [ ]:
# https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10/load_data
# labelnames = ['Airplane', 'Automobile', 'Bird', 'Cat', 'Deer', 'Dog', 'Frog', 'Horse', 'Ship', 'Truck']
labelnames = ['비행기', '자동차', '새', '고양이', '사슴',
              '개', '개구리', '말', '배', '트럭']
print(f'데이터의 레이블은 {len(labelnames)}개 이며, 데이터셋에 포함되어 있지 않으므로 설명서에서 확인해야 합니다.')
print('레이블 번호와 레이블: ')
for i in range(0, len(labelnames)):
    print(f'{i}: {labelnames[i]}')

In [ ]:
print(f'학습 이미지 형태: {train_images.shape}')
print(f'학습 레이블 형태: {train_labels.shape}')

In [ ]:
print(f'테스트 이미지 형태: {test_images.shape}')
print(f'테스트 레이블 형태: {test_labels.shape}')

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot()
axm = ax.imshow(train_images[0])
fig.colorbar(axm)
fig.suptitle('학습용 예제 이미지', fontproperties=nanum_prop, fontsize=24)
ax.set_title(f'레이블: {labelnames[train_labels[0][0]]}', fontproperties=nanum_prop, fontsize=16)
ax.grid(False)

In [ ]:
print('인공 신경망은 주로 -1.0 ~ 1.0 사이의 값을 받습니다.')
print('따라서 이미지를 255로 나누어줍니다.')
train_images = train_images / 255.0
test_images = test_images / 255.0

In [ ]:
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot()
axm = ax.imshow(train_images[0])
fig.colorbar(axm)
fig.suptitle('학습용 예제 이미지', fontproperties=nanum_prop, fontsize=24)
ax.set_title(f'레이블: {labelnames[train_labels[0][0]]}', fontproperties=nanum_prop, fontsize=16)
ax.grid(False)

In [ ]:
fig = plt.figure(figsize=(12, 12/5*2))
fig.set_facecolor('white')
for i in range(10):
    ax = fig.add_subplot(2, 5, i+1)
    for j in range(10000, len(train_images)):
        if train_labels[j] == i:
            break
    ax.imshow(train_images[j], cmap=plt.cm.binary)
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_title(f'[{train_labels[j][0]}] {labelnames[train_labels[j][0]]}', 
                 fontproperties=nanum_prop, fontsize=12)

In [ ]:
print('모델 생성')
model = tf.keras.Sequential()
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(train_images.shape[1:])))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), padding='same'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), padding='same'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(tf.keras.layers.MaxPooling2D((2, 2), strides=(1, 1), padding='same'))
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(10, activation='softmax'))
model.summary()

In [ ]:
print('모델 컴파일')
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9),
              # optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
history = {}
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

In [ ]:
epochs = 300
results = model.fit(train_images, train_labels, 
                    validation_data=(test_images, test_labels),
                    # callbacks=[callback],
                    epochs=epochs, verbose='auto')
tmp = history.get('loss', [])
for loss in results.history['loss']:
    tmp.append(loss)
history['loss'] = tmp
tmp = history.get('accuracy', [])
for accuracy in results.history['accuracy']:
    tmp.append(accuracy)
history['accuracy'] = tmp
tmp = history.get('val_loss', [])
for loss in results.history['val_loss']:
    tmp.append(loss)
history['val_loss'] = tmp
tmp = history.get('val_accuracy', [])
for accuracy in results.history['val_accuracy']:
    tmp.append(accuracy)
history['val_accuracy'] = tmp

In [ ]:
x = range(1, len(history['val_accuracy'])+1)
y = history['val_accuracy']

In [ ]:
fig = plt.figure(figsize=(8*1, 6*1))
fig.set_facecolor('white')
ax = fig.add_subplot()
ax.plot(x, y, color='blue')
ax.set_ylim((0, 1))
ax.tick_params(axis='both', labelsize='large')
ax.set_xlabel('Rounds', fontsize='x-large')
ax.set_ylabel('Accuracy', fontsize='x-large')